In [2]:
with open('input.txt', 'r') as file:
    links = file.readlines()

# Step 2: Remove duplicates by converting the list to a set
unique_links = set(links)

# Step 3: Write the unique links back to a new file
with open('unique_links.txt', 'w') as file:
    for link in sorted(unique_links):
        file.write(link)

print("Duplicates removed and unique links saved to 'unique_links.txt.")

Duplicates removed and unique links saved to 'unique_links.txt.


In [1]:
!pip install Flask
!pip install pyngrok


  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [3]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Sample in-memory data store
weblinks = [
    {"id": 1, "url": "https://en.wikipedia.org/wiki/(300163)_2006_VW139", "bin": "Bin0", "location": "Loc1", "category_vector": [0, 0, 3, 0, 5]},
    {"id": 2, "url": "https://en.wikipedia.org/wiki/.dwg", "bin": "Bin3", "location": "Loc4", "category_vector": [1, 0, 0, 0, 0]}
]

@app.route('/weblinks', methods=['GET'])
def get_weblinks():
    category_vector = request.args.getlist('category_vector')
    if category_vector:
        # Convert to list of integers
        category_vector = list(map(int, category_vector))
        result = [link for link in weblinks if link['category_vector'] == category_vector]
    else:
        result = weblinks
    return jsonify(result)

@app.route('/weblinks', methods=['POST'])
def add_weblink():
    new_weblink = request.json
    weblinks.append(new_weblink)
    return jsonify(new_weblink), 201


In [11]:
from threading import Thread
def run():
    app.run(port=8000)
# Start the Flask app in a separate thread
thread = Thread(target=run)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [6]:
pip install flask pyngrok

Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyngrok import ngrok

# Set up ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

t=2024-10-01T23:21:58+0530 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-10-01T23:21:58+0530 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2024-10-01T23:21:58+0530 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [7]:
pip install tenseal

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import tenseal as ts

# Step 1: Encryption Setup
def setup_fhe():
    # Create TenSEAL BFV context
    context = ts.context(
        ts.SCHEME_TYPE.BFV,
        poly_modulus_degree=8192,
        plain_modulus=1032193
    )
    context.generate_galois_keys()
    return context

# Step 2: Client encrypts the query vector
def encrypt_query(context):
    query = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
    enc_query = ts.bfv_vector(context, query)  # Encrypt the entire query vector as a packed BFV vector
    return enc_query

# Step 3: Server maps links to numbers
def link_mapping():
    links = ["Link1", "Link2", "Link3", "Link4", "Link6", "Link5", "Link7", "Link8", "Link9", "Link10"]
    link_to_num = {link: i+1 for i, link in enumerate(links)}  # Create a map of links to integers
    num_to_link = {i+1: link for i, link in enumerate(links)}  # Create a reverse map of numbers to links
    return link_to_num, num_to_link

# Step 4: Server encrypts the link mappings (as numbers)
def encrypt_links(context, link_to_num):
    links_as_numbers = [link_to_num[f"Link{i+1}"] for i in range(10)]  # Links mapped to numbers
    enc_links = ts.bfv_vector(context, links_as_numbers)  # Encrypt the numbers as a packed vector
    return enc_links

# Step 5: Server performs homomorphic multiplication
def homomorphic_multiplication(enc_query, enc_links):
    mult_result = enc_query * enc_links  # Element-wise homomorphic multiplication
    return mult_result

# Step 6: Client decrypts the result and maps it back to the string link
def decrypt_result(context, result, num_to_link):
    decrypted = result.decrypt()  # Decrypt the result vector
    decrypted_link_num = decrypted[4]  # Get the value at position 5 (index 4 in 0-based indexing)

    # Map the decrypted number back to the original link string
    link_at_position_5 = num_to_link[decrypted_link_num]

    return link_at_position_5

# Putting everything together
def main():
    # Encryption setup
    context = setup_fhe()

    # Client-side encryption of the query vector
    enc_query = encrypt_query(context)

    # Server-side mapping of links to numbers
    link_to_num, num_to_link = link_mapping()

    # Server-side encryption of the mapped links (numbers)
    enc_links = encrypt_links(context, link_to_num)

    # Server performs homomorphic multiplication
    mult_result = homomorphic_multiplication(enc_query, enc_links)

    # Client decrypts the result and maps it back to the original link
    link_at_position_5 = decrypt_result(context, mult_result, num_to_link)

    print(f"The link at position 5 is: {link_at_position_5}")

if __name__ == "__main__":
    main()



AttributeError: module 'tenseal' has no attribute 'Context'